In [1]:
import numpy as np
import pandas as pd
import psycopg2

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
# create a database connection
sqluser = 'mimic'
dbname = 'mimic'
schema_name = 'mimiciii'

cur = None

In [13]:
if cur: 
    cur.close()
    con.close()

con = psycopg2.connect(dbname = dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

In [14]:
query = \
"""
SELECT * FROM echo_features_labs; 
"""
el = pd.read_sql_query(query, con)
el.head()

,row_id,icustay_id,hadm_id,subject_id,charttime,label
0,67326,216859,111970,21,2135-02-03 05:04:00,White Blood Cells
1,67326,216859,111970,21,2135-02-03 05:13:00,Lactate
2,67326,216859,111970,21,2135-02-03 18:54:00,Lactate
3,67326,216859,111970,21,2135-02-03 18:54:00,pH
4,67326,216859,111970,21,2135-02-04 04:08:00,Albumin


In [15]:
len(el['hadm_id'].unique())

1931

Number of unique hospital admission IDs with at least 1 lab: 

In [16]:
el[['label', 'hadm_id']].drop_duplicates().groupby('label')['hadm_id'].count()

label
Albumin                          1146
C-Reactive Protein                 81
CK-MB Index                       139
Creatine Kinase (CK)              759
Creatine Kinase, MB Isoenzyme     655
Creatinine                       1931
Estimated GFR (MDRD equation)     737
Hematocrit, Calculated           1580
Hemoglobin                       1580
Lactate                          1730
NTproBNP                           70
Sedimentation Rate                 79
Troponin I                         25
Troponin T                        544
WBC Count                          18
White Blood Cells                1930
pH                               1874
Name: hadm_id, dtype: int64

Look at the labs 

Search lab terms in d_items: 